In [1]:
import numpy as np
import cv2 

In [2]:
def eye_detect(path):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_casecade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    ff = np.fromfile(path, np.uint8)
    img = cv2.imdecode(ff, cv2.IMREAD_UNCHANGED)
    img = cv2.resize(img, dsize=(0, 0), fx=1.0, fy=1.0, interpolation=cv2.INTER_LINEAR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 얼굴 찾는 코드
    faces = face_cascade.detectMultiScale(gray, 1.2, 5)
    
    # 눈 찾는 코드
    eyes=[]
    for (x,y,w,h) in faces:
        # cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_casecade.detectMultiScale(roi_gray)
    
    return img, eyes, roi_color

In [3]:
img, eyes, roi_color = eye_detect('./Data/Image/02.jpg')

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
img, eyes, roi_color = eye_detect('./Data/Image/02.jpg')

x1, x2, x3, x4 =eyes[0]
y1, y2, y3, y4=eyes[1]

left=roi_color[x1:x1+x3,x2:x2+x4]
right=roi_color[y2:y2+y4,y1:y1+y3]

cv2.imshow('img2', left)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow('img2', right)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# 눈만 저장
img2, eyes2, roi_color2 = eye_detect('./Data/Image/face.jpg')

x11, x12, x13, x14 =eyes2[0]
y11, y12, y13, y14=eyes2[1]

left2=roi_color2[x11:x11+x13,x12:x12+x14]
right2=roi_color2[y12:y12+y14,y11:y11+y13]

cv2.imwrite('./Data/Image/Test/left.jpg',left2)
cv2.imwrite('./Data/Image/Test/right.jpg',right2)

True

In [7]:
# 알파 채널을 마스크 영상으로 이용

left = cv2.imread('./Data/Image/Test/left.jpg', cv2.IMREAD_UNCHANGED)
left = cv2.resize(left, (0, 0), fx=4, fy=4, interpolation=cv2.INTER_NEAREST)
left = cv2.resize(left, (32, 32))

right = cv2.imread('./Data/Image/Test/right.jpg', cv2.IMREAD_UNCHANGED)
right = cv2.resize(right, (0, 0), fx=4, fy=4, interpolation=cv2.INTER_NEAREST)
right = cv2.resize(right, (32, 32))


mask1 = left[:, :, -1]    # mask는 알파 채널로 만든 마스크 영상
src1 = left[:, :, 0:3]    # src는 b, g, r 3채널로 구성된 컬러 영상

mask2 = right[:, :, -1]    # mask는 알파 채널로 만든 마스크 영상
src2 = right[:, :, 0:3]    # src는 b, g, r 3채널로 구성된 컬러 영상

dst = cv2.imread('./Data/Image/02.jpg', cv2.IMREAD_COLOR)

h1, w1 = left.shape[:2]
h2, w2 = right.shape[:2]

crop1 = dst[0:h1, 0:w1]    # src, mask와 같은 크기의 부분 영상 추출
crop2 = dst[0:h2, 0:w2]    # src, mask와 같은 크기의 부분 영상 추출

cv2.copyTo(left, mask1, crop1)
cv2.copyTo(right, mask2, crop2)

cv2.imshow('dst', dst)

cv2.waitKey() 
cv2.destroyAllWindows()